In [2]:

import wandb
import pandas as pd
import os
import numpy as np

In [3]:
api = wandb.Api()

entity = "evangeorgerex"
project = "fwal"

#### Utilities

In [4]:
results_dir = r'/home/er647/projects/feature-wise-active-learning/results'

In [5]:
def get_tables(averaged_data):
    # Pivoting for 'bestmodel_train/F1_weighted'
    train_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_train/F1_weighted')
    # print("Training F1 Weighted Averages:")
    # print(train_pivot)
    # print("\n")  # Add some space between tables

    # Pivoting for 'bestmodel_valid/F1_weighted'
    valid_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_valid/F1_weighted')
    # print("Validation F1 Weighted Averages:")
    # print(valid_pivot)
    # print("\n")  # Add some space between tables

    # Pivoting for 'bestmodel_test/F1_weighted'
    test_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_test/F1_weighted')
    # print("Test F1 Weighted Averages:")
    # print(test_pivot)
    for df in [train_pivot, valid_pivot, test_pivot]:
        # df.drop('MNIST', axis=1, inplace=True)
        # df.drop('gisette', axis=1, inplace=True) 
        df.drop('poly_binarised_decimalised_synth', axis=1, inplace=True)

    def highlight_max(s):
        '''
        Highlight the maximum in a Series by changing font color to red and making it bold.
        '''
        is_max = s == s.max()
        return ['color: green; font-weight: bold' if v else '' for v in is_max]


    table_styles = [
        {'selector': 'th',
        'props': [('background', '#606060'), ('color', 'white')]
        }
    ]
    
    train_table = train_pivot.style.set_table_styles(table_styles).set_caption("Training F1 Weighted Averages").apply(highlight_max)
    valid_table = valid_pivot.style.set_table_styles(table_styles).set_caption("Valid F1 Weighted Averages").apply(highlight_max)
    test_table = test_pivot.style.set_table_styles(table_styles).set_caption("Test F1 Weighted Averages").apply(highlight_max)
    
    return train_table, valid_table, test_table

In [6]:
def fetch_runs(entity, project, sweep_ids):
    api = wandb.Api()
    runs_data = []

    for sweep_id in sweep_ids:
        sweep_runs = api.sweep(f"{entity}/{project}/{sweep_id}").runs
        for run in sweep_runs:
            # Check if the run's state is 'finished'
            if run.state == "finished" or sweep_id == "cwhd81fy":
                dataset_name = run.config.get("dataset", "Unknown")
                if dataset_name in ["poly_binarised_decimalised_mod10_synth", "simple_trig_synth", "Unknown"]:
                    continue
                
                if run.config.get("as_MLP_baseline", False):
                    model_name = "MLP_same_capacity"
                else:
                    model_name = run.config.get("model", "Unknown")
                    
                    if model_name == "fwal":
                        model_name = "F-Act"
                        
                    # if run.config.get("hierarchical", False):
                    #     model_name = "F-Act hierarchical"
                    #     if run.config.get("share_mask", False):
                    #         model_name = "F-Act hierarchical shared"
                    #         if run.config.get("sigmoid_loss", False):
                    #             model_name = "F-Act hierarchical shared L_sig"
                    # if sweep_id == "x9s9n05b":
                    #     model_name = "F-Act hierarchical TTI (old)"
                    # if sweep_id in ["9npryg91", "7i2o46ri"]:
                    #     model_name = "F-Act hierarchical TTI"
                run_data = {
                    "model": model_name,  # Assuming model name is stored in 'model_name'
                    "dataset": dataset_name,  # Assuming dataset name is stored in 'dataset_name'
                    "seed": run.config.get("seed_model_init", None),
                    "bestmodel_train/F1_weighted": run.summary.get("bestmodel_train/F1_weighted", float('nan')),
                    "bestmodel_valid/F1_weighted": run.summary.get("bestmodel_valid/F1_weighted", float('nan')),
                    "bestmodel_test/F1_weighted": run.summary.get("bestmodel_test/F1_weighted", float('nan')),
                }
                
                
                # Specific hyperparameters
                if model_name == "lasso":
                    run_data["lasso_C"] = run.config.get("lasso_C")
                    run_data["lasso_l1_ratio"] = run.config.get("lasso_l1_ratio")
                elif model_name == "rf":
                    run_data["rf_max_depth"] = run.config.get("rf_max_depth")
                elif model_name in ["cae", "supervised_cae"]:
                    run_data["CAE_neurons_ratio"] = run.config.get("CAE_neurons_ratio")
                elif model_name == "F-Act hierarchical TTI":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act hierarchical TTI (old)":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act":
                    if sweep_id in ["bga4xu49"]:
                        run_data['dropout'] = 0.2
                        run_data["lr"] = run.config.get("lr")
                        run_data["num_hidden"] = run.config.get("num_hidden")
                        run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                    else:
                        # sweep_id == "7i2o46ri":
                        run_data['dropout'] = 0.0
                        run_data["lr"] = run.config.get("lr")
                        run_data["num_hidden"] = 4
                        run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "xgboost":
                    run_data["xgb_eta"] = run.config.get("xgb_eta")
                    run_data["xgb_max_depth"] = run.config.get("xgb_max_depth")
                elif model_name in ["MLP_same_capacity", "SEFS"]:
                    run_data["lr"] = run.config.get("lr")
                    run_data["num_hidden"] = run.config.get("num_hidden")
                    
                    
                    
                runs_data.append(run_data)
                if run_data['model'] == 'F-Act':
                    
                    opt_tti_valid, opt_tti_test, full_tti_valid, full_tti_test = get_tti_results(run)
                    opt_tti_run_data = run_data.copy()
                    opt_tti_run_data['bestmodel_valid/F1_weighted'] = opt_tti_valid
                    opt_tti_run_data['bestmodel_test/F1_weighted'] = opt_tti_test
                    opt_tti_run_data['model'] = 'F-Act (optimal TTI)'
                    runs_data.append(opt_tti_run_data)
                    
                    full_tti_run_data = run_data.copy()
                    full_tti_run_data['bestmodel_valid/F1_weighted'] = full_tti_valid
                    full_tti_run_data['bestmodel_test/F1_weighted'] = full_tti_test
                    full_tti_run_data['model'] = 'F-Act (full TTI)'
                    runs_data.append(full_tti_run_data)
                
    return runs_data

def process_runs_data(runs_data):
    df = pd.DataFrame(runs_data)
    
    # Models with hyperparameter sweeps
    models_with_hyperparams = [
        'lasso', 'rf', 'cae', 'supervised_cae', 'F-Act hierarchical TTI', 'F-Act hierarchical TTI (old)',
        'F-Act', 'MLP_same_capacity', 'SEFS', 'xgboost', 
    ]
    best_hyperparams = {}
    
    models_stored =  df['model'].unique()
    for model in models_with_hyperparams:
        if model not in models_stored:
            continue
        model_df = df[df['model'] == model]

        # For the 'F-Act' model, first find the best seed for each combination of hyperparameters
        if model == 'F-Act':
            max_seed_df = model_df.groupby(['dataset', 'sparsity_regularizer_hyperparam', 'lr', 'num_hidden', 'dropout', 'seed'])['bestmodel_valid/F1_weighted'].idxmax()
            model_df = model_df.loc[max_seed_df]
        
        # Further aggregate to find the best hyperparameter combination per dataset
        group_columns = {
            'lasso': ['dataset', 'lasso_C', 'lasso_l1_ratio'],
            'rf': ['dataset', 'rf_max_depth'],
            'cae': ['dataset', 'CAE_neurons_ratio'],
            'supervised_cae': ['dataset', 'CAE_neurons_ratio'],
            'F-Act hierarchical TTI': ['dataset', 'sparsity_regularizer_hyperparam'],
            'F-Act hierarchical TTI (old)': ['dataset', 'sparsity_regularizer_hyperparam'],
            'F-Act': ['dataset', 'sparsity_regularizer_hyperparam', 'lr', 'num_hidden', 'dropout'],
            'xgboost': ['dataset', 'xgb_eta', 'xgb_max_depth'],
            'MLP_same_capacity': ['dataset', 'lr', 'num_hidden'],
            'SEFS': ['dataset', 'lr', 'num_hidden']
        }

        best_combinations = model_df.groupby(group_columns[model])['bestmodel_valid/F1_weighted'].mean().reset_index()
        # Identify the best hyperparameters for each dataset
        best_combinations = best_combinations.loc[best_combinations.groupby('dataset')['bestmodel_valid/F1_weighted'].idxmax()]
        best_hyperparams[model] = best_combinations
        
        # Create a tuple representation for easy comparison
        best_combinations_tuples = best_combinations[group_columns[model]].apply(tuple, axis=1)

        # Filter the original df to include only runs with the best hyperparameters for the given model
        df = df[~((df['model'] == model) & ~df[group_columns[model]].apply(tuple, axis=1).isin(best_combinations_tuples)) | (df['model'] != model)]
    
    # Now, group by model and dataset to average across seeds for the final analysis
    final_grouped = df.groupby(['model', 'dataset']).mean().reset_index()
    
    # Printing best hyperparameters for each model
    # for model, params_df in best_hyperparams.items():
    #     print(f"Best hyperparameters for {model}:")
    #     print(params_df)
    #     print("\n")
    
    return final_grouped, best_hyperparams

# F-Act

In [7]:
def get_tti_results(run):
    '''
    returns:
        float: optimal test-time intervention validation f1
        float: optimal test-time intervention test f1
        float: full test-time intervention validation f1
        float: full test-time intervention test f1
    '''

    f1_weighted = []
    num_features = []

    history_dict = {}

    # Iterate through the run history to extract the desired metrics
    keys = [
        "tti_valid_metrics.F1_weighted", "tti_test_metrics.F1_weighted", "num_additional_features"
        ]
    for row in run.scan_history(keys=keys):
        # print('.')
        # Check if both keys exist in the current row
        for key in keys:
            if key in row and row[key] is not None:
                if key not in history_dict:
                    history_dict[key] = []
                history_dict[key].append(row[key])
            else:
                # print(key)
                continue
            
    max_index = np.argmax(history_dict['tti_valid_metrics.F1_weighted'])
    opt_tti_valid = history_dict['tti_valid_metrics.F1_weighted'][max_index]
    opt_tti_test = history_dict['tti_test_metrics.F1_weighted'][max_index]
    
    full_tti_valid = history_dict['tti_valid_metrics.F1_weighted'][-1]
    full_tti_test = history_dict['tti_test_metrics.F1_weighted'][-1]
    
    return opt_tti_valid, opt_tti_test, full_tti_valid, full_tti_test 
    

In [10]:
run_ids = [
    'bfi5l6qu',
    'ihpfigr4',
    'rhrqy9bb',
    'sj64pl05',
    'dtv0wjox',
    'gp378vp4',
    'vqcdnueq',
    'b8nrupvc',
    'rcacu3yt',
    'swbk9dsz',
    'sawnisk5',
    'dqq6q5ej',
    '33iwvq59',
    'ko5c4p3v',
    'h4m5vvcm',
    'zyuohsyk',    
    '0msrkm0b',
    '3v8n9qe6',
    '3e4jm286',
    'n0mcba2f',
    '94j8ch8b',
    'qskac7qn',
]
hyperparams = ['dataset', 'sparsity_regularizer_hyperparam', 'lr', 'num_hidden', 'dropout']
metrics = ['bestmodel_train/F1_weighted', 'bestmodel_valid/F1_weighted', 'bestmodel_test/F1_weighted']
model_name = 'F-Act'

In [ ]:
assert False
# Please don't auto run the cells below.

In [11]:
runs_data = []
for run_id in run_ids:
    run = api.run(f"{entity}/{project}/{run_id}")
    
    run_data = {
    "model": 'F-Act',  # Assuming model name is stored in 'model_name'
    "dataset": run.config.get("dataset", None),  # Assuming dataset name is stored in 'dataset_name'
    "seed": run.config.get("seed_model_init", None),
    "bestmodel_train/F1_weighted": run.summary.get("bestmodel_train/F1_weighted", float('nan')),
    "bestmodel_valid/F1_weighted": run.summary.get("bestmodel_valid/F1_weighted", float('nan')),
    "bestmodel_test/F1_weighted": run.summary.get("bestmodel_test/F1_weighted", float('nan')),
    }
    
    runs_data.append(run_data)
    
    opt_tti_valid, opt_tti_test, full_tti_valid, full_tti_test = get_tti_results(run)
    opt_tti_run_data = run_data.copy()
    opt_tti_run_data['bestmodel_valid/F1_weighted'] = opt_tti_valid
    opt_tti_run_data['bestmodel_test/F1_weighted'] = opt_tti_test
    opt_tti_run_data['model'] = 'F-Act (optimal TTI)'
    runs_data.append(opt_tti_run_data)
    
    full_tti_run_data = run_data.copy()
    full_tti_run_data['bestmodel_valid/F1_weighted'] = full_tti_valid
    full_tti_run_data['bestmodel_test/F1_weighted'] = full_tti_test
    full_tti_run_data['model'] = 'F-Act (full TTI)'
    runs_data.append(full_tti_run_data)
    

In [15]:
fact_df = pd.DataFrame(runs_data).groupby(['dataset', 'model']).mean().reset_index()

In [19]:
train_table, valid_table, test_table = get_tables(fact_df)

In [20]:
train_table

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.999479
F-Act (full TTI),1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.999479
F-Act (optimal TTI),1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.999479


In [21]:
valid_table

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.992910,0.926447,0.846068,0.968084,0.599168,0.728721,0.984575
F-Act (full TTI),0.991810,0.942990,0.854802,0.962459,0.575430,0.722341,0.995381
F-Act (optimal TTI),0.995282,0.942990,0.879263,0.966608,0.602687,0.741475,0.996920


In [22]:
test_table

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.986039,0.919675,0.831719,0.968292,0.602923,0.722480,0.978486
F-Act (full TTI),0.982608,0.928581,0.913124,0.960261,0.599476,0.702223,0.989209
F-Act (optimal TTI),0.988419,0.928581,0.898703,0.959526,0.598131,0.729048,0.989209


In [56]:
fact_df.to_csv(os.path.join(results_dir, 'fact.csv'))

In [82]:
fact_df

,dataset,model,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted
0,COIL20,F-Act,1.0,1.000000,0.992910,0.986039
1,COIL20,F-Act (full TTI),1.0,1.000000,0.991810,0.982608
2,COIL20,F-Act (optimal TTI),1.0,1.000000,0.995282,0.988419
3,Isolet,F-Act,1.0,0.999628,0.926447,0.919675
4,Isolet,F-Act (full TTI),1.0,0.999628,0.942990,0.928581
5,Isolet,F-Act (optimal TTI),1.0,0.999628,0.942990,0.928581
6,PBMC,F-Act,2.0,1.000000,0.846068,0.831719
7,PBMC,F-Act (full TTI),2.0,1.000000,0.854802,0.913124
8,PBMC,F-Act (optimal TTI),2.0,1.000000,0.879263,0.898703
9,USPS,F-Act,1.0,0.990255,0.968084,0.968292


In [9]:
# load fact_df from csv:
fact_df = pd.read_csv(os.path.join(results_dir, 'fact.csv'), index_col=0)

# Lasso

In [24]:
lasso_sweep_ids = [
    'rdvu3fg8', # everything except finance
    'xpxe64ee', # finance
]

In [28]:
# Fetch and process the data
lasso_runs_data = fetch_runs(entity, project, lasso_sweep_ids)

In [29]:
lasso_averaged_data, lasso_best_hyperparams = process_runs_data(lasso_runs_data)


Best hyperparameters for lasso:
                              dataset  lasso_C  lasso_l1_ratio  \
2                              COIL20       10            0.50   
21                             Isolet      100            0.25   
30                               PBMC       10            0.00   
48                               USPS       10            0.75   
62                            finance       10            0.50   
75                            madelon       10            0.00   
95                       mice_protein      100            0.00   
112  poly_binarised_decimalised_synth      100            0.50   

     bestmodel_valid/F1_weighted  
2                       0.982551  
21                      0.947942  
30                      0.873614  
48                      0.934424  
62                      0.595040  
75                      0.548075  
95                      0.964513  
112                     0.502693  




In [57]:
lasso_averaged_data.to_csv(os.path.join(results_dir, 'lasso.csv'))


In [81]:
lasso_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,lasso_C,lasso_l1_ratio
0,lasso,COIL20,1.0,1.000000,0.982551,0.982413,10.0,0.50
1,lasso,Isolet,1.0,1.000000,0.947942,0.945832,100.0,0.25
2,lasso,PBMC,1.0,1.000000,0.873614,0.892459,10.0,0.00
3,lasso,USPS,1.0,0.994983,0.934424,0.933578,10.0,0.75
4,lasso,finance,1.0,0.670358,0.595040,0.597777,10.0,0.50
5,lasso,madelon,1.0,0.803830,0.548075,0.515320,10.0,0.00
6,lasso,mice_protein,1.0,0.988156,0.964513,0.952401,100.0,0.00
7,lasso,poly_binarised_decimalised_synth,1.0,0.525506,0.502693,0.483242,100.0,0.50


In [11]:
lasso_df = pd.read_csv(os.path.join(results_dir, 'lasso.csv'), index_col=0)

# XGBoost

In [35]:
xgb_sweep_ids = [
    'yr11wqmg', # everything
]


In [36]:
# Fetch and process the data
xgb_runs_data = fetch_runs(entity, project, xgb_sweep_ids)

In [37]:
xgb_averaged_data, xgb_best_hyperparams = process_runs_data(xgb_runs_data)


In [58]:
xgb_averaged_data.to_csv(os.path.join(results_dir, 'xgb.csv'))


In [80]:
xgb_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,xgb_eta,xgb_max_depth
0,xgboost,COIL20,1.0,1.000000,0.985390,0.986075,0.1,6.0
1,xgboost,Isolet,1.0,1.000000,0.902887,0.887528,0.1,9.0
2,xgboost,PBMC,1.0,0.995177,0.898703,0.894172,0.3,3.0
3,xgboost,USPS,1.0,1.000000,0.968251,0.973689,0.3,3.0
4,xgboost,finance,1.0,0.861745,0.634798,0.588311,0.5,6.0
5,xgboost,madelon,1.0,1.000000,0.807590,0.809609,0.1,6.0
6,xgboost,mice_protein,1.0,1.000000,0.978342,0.981490,0.3,3.0
7,xgboost,poly_binarised_decimalised_synth,1.0,0.972310,0.504693,0.501841,0.1,9.0


# RF

In [101]:
rf_sweep_ids = [
    "22eckaxx", # rf baseline
    "gty91unf", # rf baseline
    "4upk4xrk", # RF finance
    'l102r3et', # PBMC   
]

In [102]:
# Fetch and process the data
rf_runs_data = fetch_runs(entity, project, rf_sweep_ids)

In [138]:
rf_averaged_data, xgb_best_hyperparams = process_runs_data(rf_runs_data)
# drop "gisette" dataset
rf_averaged_data = rf_averaged_data[(rf_averaged_data['dataset'] !='gisette' )&( rf_averaged_data['dataset'] !='MNIST') ]


In [18]:
rf_df = rf_df[(rf_df['dataset'] !='gisette' )&( rf_df['dataset'] !='MNIST') ]

In [19]:
rf_df.to_csv(os.path.join(results_dir, 'rf.csv'))


In [131]:
rf_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,rf_max_depth
0,rf,COIL20,1.0,0.986327,0.970085,0.967561,7.0
1,rf,Isolet,1.0,0.984742,0.921524,0.900926,7.0
2,rf,MNIST,1.0,0.921460,0.910721,0.911655,7.0
3,rf,PBMC,1.0,0.937396,0.869854,0.886592,7.0
4,rf,USPS,1.0,0.957755,0.940427,0.933634,7.0
5,rf,finance,1.0,0.877761,0.600702,0.619456,7.0
6,rf,gisette,1.0,0.973730,0.949999,0.955949,7.0
7,rf,madelon,1.0,0.995769,0.693069,0.671901,7.0
8,rf,mice_protein,1.0,0.999807,0.962906,0.969803,7.0
9,rf,poly_binarised_decimalised_synth,1.0,0.742473,0.506308,0.505766,7.0


# MLP

In [38]:
mlp_sweep_ids = [
    's5m2sqsv', # everything
]


In [39]:
# Fetch and process the data
mlp_runs_data = fetch_runs(entity, project, mlp_sweep_ids)

In [40]:
mlp_averaged_data, mlp_best_hyperparams = process_runs_data(mlp_runs_data)


In [59]:
mlp_averaged_data.to_csv(os.path.join(results_dir, 'mlp.csv'))


In [79]:
mlp_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,lr,num_hidden
0,MLP_same_capacity,COIL20,1.0,1.000000,0.992998,0.988311,0.0010,1.0
1,MLP_same_capacity,Isolet,1.0,0.999255,0.944965,0.934809,0.0010,2.0
2,MLP_same_capacity,PBMC,1.0,0.999349,0.898915,0.894191,0.0010,4.0
3,MLP_same_capacity,USPS,1.0,0.990137,0.969688,0.967768,0.0010,1.0
4,MLP_same_capacity,finance,1.0,0.610960,0.570507,0.570214,0.0010,2.0
5,MLP_same_capacity,madelon,1.0,0.819199,0.593281,0.571757,0.0010,1.0
6,MLP_same_capacity,mice_protein,1.0,0.999479,0.992254,0.983014,0.0001,1.0
7,MLP_same_capacity,poly_binarised_decimalised_synth,1.0,0.527624,0.503680,0.472102,0.0001,2.0


# SEFS

In [41]:
sefs_sweep_ids = [
    'n6zb5lc9', # everything
]

In [42]:
# Fetch and process the data
sefs_runs_data = fetch_runs(entity, project, sefs_sweep_ids)

In [43]:
sefs_averaged_data, sefs_best_hyperparams = process_runs_data(sefs_runs_data)


In [60]:
sefs_averaged_data.to_csv(os.path.join(results_dir, 'sefs.csv'))


In [78]:
sefs_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,lr,num_hidden
0,SEFS,COIL20,1.0,0.979644,0.963678,0.949701,0.003,1.0
1,SEFS,Isolet,1.0,0.976174,0.882264,0.886111,0.003,1.0
2,SEFS,USPS,1.0,0.938292,0.935731,0.925440,0.003,1.0
3,SEFS,finance,1.0,0.615141,0.580251,0.599271,0.003,2.0
4,SEFS,madelon,1.0,0.761723,0.642004,0.652284,0.003,2.0
5,SEFS,mice_protein,1.0,0.904336,0.860212,0.850778,0.003,1.0
6,SEFS,poly_binarised_decimalised_synth,1.0,0.374214,0.362826,0.366728,0.003,1.0


# CAE

In [47]:
cae_sweep_ids = [
    'h245pc2i', 
    'jqulrkke', # finance
    'xh18ha0v', # PBMC
]

In [48]:
# Fetch and process the data
cae_runs_data = fetch_runs(entity, project, cae_sweep_ids)

In [49]:
cae_averaged_data, sefs_best_hyperparams = process_runs_data(cae_runs_data)


In [61]:
cae_averaged_data.to_csv(os.path.join(results_dir, 'cae.csv'))


In [77]:
cae_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,CAE_neurons_ratio
0,cae,COIL20,1.0,1.000000,0.983572,0.970427,1.0
1,cae,Isolet,1.0,0.962531,0.828027,0.801414,0.8
2,cae,PBMC,1.0,0.882927,0.667353,0.680679,0.1
3,cae,USPS,1.0,0.930945,0.904831,0.904749,1.0
4,cae,finance,1.0,0.656600,0.611487,0.592564,0.7
5,cae,madelon,1.0,0.873945,0.727068,0.701883,1.0
6,cae,mice_protein,1.0,0.962683,0.894743,0.853472,0.6
7,cae,poly_binarised_decimalised_synth,1.0,0.445899,0.419617,0.434222,0.8


# Supervised CAE

In [84]:
scae_sweep_ids = [
    '8yghctpp', 
    'xclyvl0c', # finance
    'vle39arl', # PBMC
]

In [85]:
# Fetch and process the data
scae_runs_data = fetch_runs(entity, project, scae_sweep_ids)

In [86]:
scae_averaged_data, sefs_best_hyperparams = process_runs_data(scae_runs_data)


In [87]:
scae_averaged_data.to_csv(os.path.join(results_dir, 'scae.csv'))


In [88]:
scae_averaged_data

,model,dataset,seed,bestmodel_train/F1_weighted,bestmodel_valid/F1_weighted,bestmodel_test/F1_weighted,CAE_neurons_ratio
0,supervised_cae,COIL20,1.0,0.054281,0.058931,0.064637,0.1
1,supervised_cae,Isolet,1.0,0.040158,0.039760,0.036833,0.2
2,supervised_cae,PBMC,0.0,0.830378,0.832341,0.853663,0.1
3,supervised_cae,USPS,1.0,0.207287,0.212480,0.209018,0.1
4,supervised_cae,finance,1.0,0.537526,0.547139,0.544434,0.6
5,supervised_cae,madelon,1.0,0.610881,0.624305,0.618427,1.0
6,supervised_cae,mice_protein,1.0,0.191261,0.167532,0.172394,1.0
7,supervised_cae,poly_binarised_decimalised_synth,1.0,0.368529,0.368871,0.369297,0.3


# Full table

In [20]:
fact_df = pd.read_csv(os.path.join(results_dir, 'fact.csv'), index_col=0)
lasso_df = pd.read_csv(os.path.join(results_dir, 'lasso.csv'), index_col=0)
xgb_df = pd.read_csv(os.path.join(results_dir, 'xgb.csv'), index_col=0)
rf_df = pd.read_csv(os.path.join(results_dir, 'rf.csv'), index_col=0)
mlp_df = pd.read_csv(os.path.join(results_dir, 'mlp.csv'), index_col=0)
sefs_df = pd.read_csv(os.path.join(results_dir, 'sefs.csv'), index_col=0)
cae_df = pd.read_csv(os.path.join(results_dir, 'cae.csv'), index_col=0)
scae_df = pd.read_csv(os.path.join(results_dir, 'scae.csv'), index_col=0)

In [21]:
averaged_dfs = [
    fact_df,
    lasso_df,
    xgb_df,
    rf_df,
    mlp_df,
    sefs_df,
    cae_df,
    scae_df
]

results_df = pd.concat(averaged_dfs)

In [22]:
train_table, valid_table, test_table = get_tables(results_df)


In [23]:
train_table

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.999479
F-Act (full TTI),1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.999479
F-Act (optimal TTI),1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.999479
MLP_same_capacity,1.000000,0.999255,0.999349,0.990137,0.610960,0.819199,0.999479
SEFS,0.979644,0.976174,nan,0.938292,0.615141,0.761723,0.904336
cae,1.000000,0.962531,0.882927,0.930945,0.656600,0.873945,0.962683
lasso,1.000000,1.000000,1.000000,0.994983,0.670358,0.803830,0.988156
rf,0.986327,0.984742,0.937396,0.957755,0.877761,0.995769,0.999807
supervised_cae,0.054281,0.040158,0.830378,0.207287,0.537526,0.610881,0.191261


In [24]:
valid_table

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.992910,0.926447,0.846068,0.968084,0.599168,0.728721,0.984575
F-Act (full TTI),0.991810,0.942990,0.854802,0.962459,0.575430,0.722341,0.995381
F-Act (optimal TTI),0.995282,0.942990,0.879263,0.966608,0.602687,0.741475,0.996920
MLP_same_capacity,0.992998,0.944965,0.898915,0.969688,0.570507,0.593281,0.992254
SEFS,0.963678,0.882264,nan,0.935731,0.580251,0.642004,0.860212
cae,0.983572,0.828027,0.667353,0.904831,0.611487,0.727068,0.894743
lasso,0.982551,0.947942,0.873614,0.934424,0.595040,0.548075,0.964513
rf,0.970085,0.921524,0.869854,0.940427,0.600702,0.693069,0.962906
supervised_cae,0.058931,0.039760,0.832341,0.212480,0.547139,0.624305,0.167532


In [25]:
test_table

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.986039,0.919675,0.831719,0.968292,0.602923,0.722480,0.978486
F-Act (full TTI),0.982608,0.928581,0.913124,0.960261,0.599476,0.702223,0.989209
F-Act (optimal TTI),0.988419,0.928581,0.898703,0.959526,0.598131,0.729048,0.989209
MLP_same_capacity,0.988311,0.934809,0.894191,0.967768,0.570214,0.571757,0.983014
SEFS,0.949701,0.886111,nan,0.925440,0.599271,0.652284,0.850778
cae,0.970427,0.801414,0.680679,0.904749,0.592564,0.701883,0.853472
lasso,0.982413,0.945832,0.892459,0.933578,0.597777,0.515320,0.952401
rf,0.967561,0.900926,0.886592,0.933634,0.619456,0.671901,0.969803
supervised_cae,0.064637,0.036833,0.853663,0.209018,0.544434,0.618427,0.172394


In [26]:
def to_latex(df):
    latex_table = df.to_latex()
    modified_string = latex_table.replace(r"\th#606060", r"\centering")
    modified_string = modified_string.replace(r"MLP_same_capacity", r"MLP")
    # \n\\begin{tabular}
    modified_string = modified_string.replace("\n\\begin{tabular}{lrrrrrrr}", "\n\\begin{tabular}{lrrrrrrr}\n\\hline")
    modified_string = modified_string.replace("\\\\\nmodel &  &  &  &  &  &  &  \\\\\nF-Act", "\\\\\nmodel &  &  &  &  &  &  &  \\\\\n\\hline\nF-Act")
    modified_string = modified_string.replace("\n\\end{tabular}", "\n\\hline\n\\end{tabular}")


    import re

    def replace_color_and_bold(text):
        # Define a regex pattern to match the specific format
        pattern = r'\\colorgreen \\font-weightbold (\d+\.\d+)'

        # Define a function to replace each match
        def replace_match(match):
            number = match.group(1)  # This is the number captured by (\d+\.\d+)
            return r'\textcolor{green}{\textbf{' + number + '}}'

        # Use re.sub to replace all occurrences in the text
        modified_text = re.sub(pattern, replace_match, text)
        return modified_text

    # Example usage
    modified_string = replace_color_and_bold(modified_string)

    print(modified_string)

to_latex(test_table)

\begin{table}
\caption{Test F1 Weighted Averages}
\centering
\begin{tabular}{lrrrrrrr}
\hline
dataset & COIL20 & Isolet & PBMC & USPS & finance & madelon & mice_protein \\
model &  &  &  &  &  &  &  \\
\hline
F-Act & 0.986039 & 0.919675 & 0.831719 & 0.968292 & 0.602923 & 0.722480 & 0.978486 \\
F-Act (full TTI) & 0.982608 & 0.928581 & \textcolor{green}{\textbf{0.913124}} & 0.960261 & 0.599476 & 0.702223 & \textcolor{green}{\textbf{0.989209}} \\
F-Act (optimal TTI) & \textcolor{green}{\textbf{0.988419}} & 0.928581 & 0.898703 & 0.959526 & 0.598131 & 0.729048 & \textcolor{green}{\textbf{0.989209}} \\
MLP & 0.988311 & 0.934809 & 0.894191 & 0.967768 & 0.570214 & 0.571757 & 0.983014 \\
SEFS & 0.949701 & 0.886111 & nan & 0.925440 & 0.599271 & 0.652284 & 0.850778 \\
cae & 0.970427 & 0.801414 & 0.680679 & 0.904749 & 0.592564 & 0.701883 & 0.853472 \\
lasso & 0.982413 & \textcolor{green}{\textbf{0.945832}} & 0.892459 & 0.933578 & 0.597777 & 0.515320 & 0.952401 \\
rf & 0.967561 & 0.900926 & 0.88659